# Explorando as estatísticas de comércio exterior brasileiras

As bases de dados exportações e importações brasileiras podem ser encontradas no seguinte endereço:

https://www.gov.br/produtividade-e-comercio-exterior/pt-br/assuntos/comercio-exterior/estatisticas/base-de-dados-bruta

Observe que na primeira seção - Base de dados detalhada por NCM - encontramos links para os arquivos .csv para cada ano.

Crie uma função que gere uma lista com os urls de todos os anos disponíveis (1997 a 2023), tanto para importação quanto para exportação.

In [1]:
url_base = "https://balanca.economia.gov.br/balanca/bd/comexstat-bd/ncm/"

def cria_url(fluxo, ano):
    return f"{url_base}{fluxo.upper()}_{str(ano)}.csv"

In [2]:
urls_balanca = [cria_url(f, a) for f in ["exp", "imp"] for a in range(1997, 2024)]

Aproveitando os códigos da aula, baixe e salve o arquivo de exportações de 1997.

In [4]:
import os
import requests

def build_path(folder):
    current_folder = os.getcwd()
    folderpath = os.path.join(current_folder, folder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): os.makedirs(folderpath)
    return folderpath


# folder é uma subpasta do seu diretório atual
# o nome do arquivo é gerado automaticamente
def download_save_unverified(url, folder):
    
    folderpath = build_path(folder)
    filepath = os.path.join(folderpath, url.split('/')[-1])

    # bypass da validação para evitar SSLCertVerificationError
    response = requests.get(url, verify = False) 

    if response.status_code == 200:
        content = response.content

        with open(filepath, mode='wb') as csvfile:
            csvfile.write(content)

In [5]:
download_save_unverified(urls_balanca[0], "data/arquivos_exp")

C:\Anaconda3\envs\bootcamp_enap\lib\site-packages\urllib3\connectionpool.py:1052: InsecureRequestWarning: Unverified HTTPS request is being made to host 'balanca.economia.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


Importe a função **read_csv** da biblioteca **pandas** e tente abrir o arquivo baixado. 

In [6]:
from pandas import read_csv

read_csv("data/arquivos_exp/EXP_1997.csv")

,"CO_ANO;""CO_MES"";""CO_NCM"";""CO_UNID"";""CO_PAIS"";""SG_UF_NCM"";""CO_VIA"";""CO_URF"";""QT_ESTAT"";""KG_LIQUIDO"";""VL_FOB"""
0,"1997;""11"";""20079990"";""10"";""158"";""RJ"";""04"";""071..."
1,"1997;""11"";""48064000"";""10"";""063"";""SP"";""07"";""101..."
2,"1997;""11"";""02071300"";""10"";""351"";""SC"";""01"";""092..."
3,"1997;""11"";""19021100"";""10"";""586"";""MS"";""09"";""091..."
4,"1997;""11"";""90319090"";""10"";""040"";""ND"";""04"";""071..."
...,...
529810,"1997;""05"";""34021190"";""10"";""845"";""SP"";""07"";""101..."
529811,"1997;""05"";""20058000"";""10"";""158"";""GO"";""01"";""081..."
529812,"1997;""05"";""40119110"";""11"";""589"";""SP"";""04"";""081..."
529813,"1997;""05"";""02023000"";""10"";""245"";""RS"";""01"";""101..."


Algo estranho na formatação da tabela? Investigue a primeira linha do arquivo baixado e tente ajustar algum parâmetro da função **read_csv**.

In [7]:
import csv
filepath = "data/arquivos_exp/EXP_1997.csv"

with open(filepath) as csvfile:
    lines = csvfile.readlines()
    
lines[:1]

['"CO_ANO";"CO_MES";"CO_NCM";"CO_UNID";"CO_PAIS";"SG_UF_NCM";"CO_VIA";"CO_URF";"QT_ESTAT";"KG_LIQUIDO";"VL_FOB"\n']

In [8]:
read_csv("data/arquivos_exp/EXP_1997.csv", sep = ";")

,CO_ANO,CO_MES,CO_NCM,CO_UNID,CO_PAIS,SG_UF_NCM,CO_VIA,CO_URF,QT_ESTAT,KG_LIQUIDO,VL_FOB
0,1997,11,20079990,10,158,RJ,4,717700,0,200,540
1,1997,11,48064000,10,63,SP,7,1010900,0,58792,90186
2,1997,11,2071300,10,351,SC,1,925100,0,57165,20767
3,1997,11,19021100,10,586,MS,9,910600,0,12,9
4,1997,11,90319090,10,40,ND,4,717700,0,6,250
...,...,...,...,...,...,...,...,...,...,...,...
529810,1997,5,34021190,10,845,SP,7,1015300,0,830,1153
529811,1997,5,20058000,10,158,GO,1,817800,0,192,340
529812,1997,5,40119110,11,589,SP,4,817700,1,99,1680
529813,1997,5,2023000,10,245,RS,1,1010500,0,22102,68583


# IBGE - Frequência de nomes


A API do IBGE permite a consulta, com base em dados do Censo, da frequência de nomes por sexo, localidade e década de nascimento. Consulte sua documentação em:

https://servicodados.ibge.gov.br/api/docs/nomes?versao=2


Crie uma função que busque a frequência de um dado nome, agrupando por UF, e retorne um dicionário onde a chave é o nome da UF e o valor é a frequência do nome. Inclua um parâmetro na função para alterar a frequência pela proporção (frequência por 100.000 habitantes).

*dica*: você pode consultar a correspondência entre código e nome da UF pelo endpoint "https://servicodados.ibge.gov.br/api/v1/localidades/estados".

In [21]:
estados = requests.get("https://servicodados.ibge.gov.br/api/v1/localidades/estados").json()
codigo_estado = {str(i.get("id")): i.get("nome") for i in estados}

In [24]:
def cria_ranking_nome_estado(nome, metrica = "frequencia"):
    nome_query = f"https://servicodados.ibge.gov.br/api/v2/censos/nomes/{nome}?groupBy=UF"
    resp = requests.get(nome_query).json()
    ranking = {codigo_estado[nome.get("localidade")]: nome.get("res")[0].get(f"{metrica}") 
               for nome in resp}
    
    return ranking